# Comparing Bi-Encoders and Cross-Encoders on Scifact Dataset
**E5 Bi-Encoder, NFCorpus**

This notebook shows how to:
1. Load the nfcorpus dataset (corpus, queries, qrels)
2. Perform bi-encoder retrieval using FAISS HNSW.
3. Save or inspect intermediate variables (embeddings, top-k results).
4. Rerank the top-k results with a cross-encoder.
5. Compare the evaluations at each step.


In [2]:
# Cell 0: Select the correct kernel
!jupyter kernelspec list

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Available kernels:
  wse-env    /home/dl5214/.local/share/jupyter/kernels/wse-env
  python3    /home/shaoyu/anaconda3/share/jupyter/kernels/python3


# Part I: Preparations
1. Import packages.
2. Load data.
3. Prepare the queries for evaluation.

In [1]:
# Cell 1: Imports and Logging
import json
import logging
import torch
import faiss
import pytrec_eval
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from tqdm import tqdm
import os
import h5py

logging.basicConfig(level=logging.INFO)
print('Imports complete.')

Imports complete.


/home/dl5214/.conda/envs/wse-env/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dl5214/.conda/envs/wse-env/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any fe

In [14]:
torch.cuda.empty_cache()

In [2]:
# Cell 2: Data Loading Functions
def load_scifact_corpus(corpus_file):
    """
    Loads the Scifact corpus from a JSONL file.
    Each line is a JSON object containing '_id' and 'text'.
    Returns lists of document IDs and document texts.
    """
    doc_ids = []
    doc_texts = []
    with open(corpus_file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            doc_id = data["_id"]
            text_str = data.get("text", "")
            doc_ids.append(doc_id)
            doc_texts.append(text_str)
    return doc_ids, doc_texts

def load_scifact_queries(queries_file):
    """
    Loads the Scifact queries from a JSONL file.
    Each line is a JSON object containing '_id' and 'text'.
    Returns lists of query IDs and query texts.
    """
    q_ids = []
    q_texts = []
    with open(queries_file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            q_ids.append(data["_id"])
            q_texts.append(data.get("text", ""))
    return q_ids, q_texts

def load_qrels(qrels_file):
    """
    Loads the qrels from a TSV file.
    The first line is a header: 'query-id    corpus-id    score'.
    Returns a dictionary: qrels[query_id][doc_id] = relevance_score.
    """
    qrels = {}
    with open(qrels_file, "r", encoding="utf-8") as f:
        # Skip the header line:
        next(f)
        for line in f:
            line = line.strip()
            if not line:
                continue
            qid, did, rel = line.split()
            rel = int(rel)
            if qid not in qrels:
                qrels[qid] = {}
            qrels[qid][did] = rel
    return qrels

print('Data loading functions ready.')


Data loading functions ready.


In [3]:
# Cell 3: Metric Evaluation Function
# We'll define a single function we can call multiple times,
# whether for the bi-encoder alone or after cross-encoder reranking.

def evaluate_results(qrels, results, k_values=None):
    """
    Evaluates retrieval results using pytrec_eval.
    results: dict of dict => results[qid][doc_id] = score
    qrels: dict of dict => qrels[qid][doc_id] = relevance
    k_values: list of cutoff values (e.g. [5,10,20])
    """
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {"map", "ndcg_cut", "recall", "P"})
    scores = evaluator.evaluate(results)

    if k_values is None:
        # If k_values not specified, use typical cutoffs
        k_values = [5, 10, 20, 30, 100]

    ndcg_res = {}
    recall_res = {}
    prec_res = {}
    mrr_vals = []

    # Compute mean average precision (MAP) across all queries
    map_vals = [scores[qid]["map"] for qid in scores]
    map_res = np.mean(map_vals)

    # Compute NDCG@k, Recall@k, P@k
    for k in k_values:
        ndcg_vals = []
        recall_vals = []
        prec_vals = []
        for qid in scores:
            ndcg_key = f"ndcg_cut_{k}"
            recall_key = f"recall_{k}"
            prec_key = f"P_{k}"
            # If the metric isn't found for some reason, skip
            if ndcg_key in scores[qid]:
                ndcg_vals.append(scores[qid][ndcg_key])
            if recall_key in scores[qid]:
                recall_vals.append(scores[qid][recall_key])
            if prec_key in scores[qid]:
                prec_vals.append(scores[qid][prec_key])

        ndcg_res[f"NDCG@{k}"] = np.mean(ndcg_vals) if ndcg_vals else 0.0
        recall_res[f"Recall@{k}"] = np.mean(recall_vals) if recall_vals else 0.0
        prec_res[f"P@{k}"] = np.mean(prec_vals) if prec_vals else 0.0

    # Compute MRR (Mean Reciprocal Rank)
    for qid in results:
        ranked_docs = sorted(results[qid].items(), key=lambda x: x[1], reverse=True)
        for rank, (doc_id, _) in enumerate(ranked_docs, start=1):
            if qrels.get(qid, {}).get(doc_id, 0) > 0:  # Relevant document
                mrr_vals.append(1 / rank)
                break
        else:
            mrr_vals.append(0)  # No relevant document found in results

    mrr_res = np.mean(mrr_vals)

    return ndcg_res, map_res, recall_res, prec_res, mrr_res

print('Evaluation function ready.')

Evaluation function ready.


In [4]:
# Cell 4: Load the SciFact Data
corpus_file = "datasets/webis-touche2020/corpus.jsonl"
queries_file = "datasets/webis-touche2020/queries.jsonl"
qrels_file = "datasets/webis-touche2020/qrels/test.tsv"

doc_ids, doc_texts = load_scifact_corpus(corpus_file)
query_ids, query_texts = load_scifact_queries(queries_file)
qrels = load_qrels(qrels_file)

print(f"Loaded {len(doc_ids)} documents and {len(query_ids)} queries.")
print(f"Unique qrels: {len(qrels)}")

Loaded 382545 documents and 49 queries.
Unique qrels: 49


In [5]:
# Cell 5: Filter queries to those that appear in qrels
filtered_query_ids = [qid for qid in query_ids if qid in qrels]
filtered_query_texts = [query_texts[query_ids.index(qid)] for qid in filtered_query_ids]

print(f"Filtered queries for evaluation: {len(filtered_query_ids)}")


Filtered queries for evaluation: 49


# Part II: Bi-Encoder Retrieval with FAISS HNSW

We'll:
1. Load a SentenceTransformer (E5) model.
2. Encode all documents with prefix **"passage: "**.
3. Build a FAISS HNSW index.
4. Encode queries with prefix **"query: "**.
5. Perform approximate nearest neighbor (ANN) search.
6. Evaluate results.


In [6]:
# Cell 6: Bi-Encoder Setup
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

# Set the device to "cuda" if available, otherwise fallback to "cpu"
device = "cuda" if cuda_available else "cpu"

# Load the Bi-Encoder model on the specified device
bi_model = SentenceTransformer("intfloat/e5-base-v2", device=device)
print(f"Bi-Encoder model loaded on device: {device}")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/e5-base-v2


CUDA available: True
Bi-Encoder model loaded on device: cuda


In [7]:
# Cell 7: Encode Corpus with E5 (prefix 'passage: ')
# We'll store these embeddings in a variable so we can reuse or inspect them.
corpus_embeddings = bi_model.encode([
    "passage: " + text for text in doc_texts
],
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Corpus embeddings shape:", corpus_embeddings.shape)

Batches:   0%|          | 0/5978 [00:00<?, ?it/s]

Corpus embeddings shape: (382545, 768)


In [10]:
# Directory path to save embeddings
embedding_dir = "./embeddings"
os.makedirs(embedding_dir, exist_ok=True)  # Create directory if it doesn't exist

# Save the embeddings as an HDF5 file
hdf5_file = os.path.join(embedding_dir, "e5_corpus_embeddings_touche2020.h5")
with h5py.File(hdf5_file, "w") as f:
    # Use gzip compression to reduce file size
    f.create_dataset("embeddings", data=corpus_embeddings, compression="gzip")
    f.create_dataset("shape", data=corpus_embeddings.shape)  # Store shape for later use

print(f"Corpus embeddings saved in HDF5 format to: {hdf5_file}")

Corpus embeddings saved in HDF5 format to: ./embeddings/e5_corpus_embeddings_touche2020.h5


In [7]:
# Filepath to the saved embeddings
embedding_dir = "./embeddings"
hdf5_file = os.path.join(embedding_dir, "e5_corpus_embeddings_touche2020.h5")

# Load the embeddings from the HDF5 file
with h5py.File(hdf5_file, "r") as f:
    corpus_embeddings = f["embeddings"][:]  # Load the embeddings dataset
    embeddings_shape = tuple(f["shape"][:])  # Load the shape

print("Corpus embeddings loaded.")
print(f"Shape of loaded embeddings: {embeddings_shape}")

Corpus embeddings loaded.
Shape of loaded embeddings: (382545, 768)


In [8]:
# Cell 8: Build the FAISS HNSW Index
dimension = corpus_embeddings.shape[1]
M = 15  # number of connections per node in HNSW
efConstruction = 300
efSearch = 1000

logging.info("Building HNSW index...")
index_hnsw = faiss.IndexHNSWFlat(dimension, M)
index_hnsw.hnsw.efConstruction = efConstruction
index_hnsw.hnsw.efSearch = efSearch

# Add corpus embeddings to index
index_hnsw.add(corpus_embeddings)

# If GPU available, move index to GPU
if torch.cuda.is_available():
    logging.info("Moving HNSW index to GPU...")
    faiss_res = faiss.StandardGpuResources()
    index_hnsw = faiss.index_cpu_to_gpu(faiss_res, 0, index_hnsw)

print("HNSW index ready.")

INFO:root:Building HNSW index...
INFO:root:Moving HNSW index to GPU...


HNSW index ready.


In [9]:
# Cell 9: Encode Queries with E5 (prefix 'query: ')
# We'll store query embeddings so we can re-run or examine.

query_embeddings = bi_model.encode([
    "query: " + text for text in filtered_query_texts
],
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)
print("Query embeddings shape:", query_embeddings.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query embeddings shape: (49, 768)


## Perform ANN Search and Evaluate (Bi-Encoder Only)

We'll retrieve the top-k docs from FAISS for each query, then convert them to a `results` dict for evaluation.

In [10]:
# Cell 10: Bi-Encoder ANN Search
top_k = 500
logging.info("Searching query embeddings in HNSW index...")
D, I = index_hnsw.search(query_embeddings, top_k)

# Convert results to pytrec_eval format
# results[qid][docid] = score
bi_encoder_results = {}
for q_idx, qid in enumerate(filtered_query_ids):
    bi_encoder_results[qid] = {}
    for rank in range(top_k):
        doc_idx = I[q_idx, rank]
        # for a distance D[q_idx, rank], we can invert or just use negative distance
        # Some prefer 1/dist, or -dist, or use it directly.
        # We'll do a small trick: score = 1 / distance.
        # If distance is 0, we can handle that or set a high value.
        dist = D[q_idx, rank]
        score = 1/float(dist) if dist != 0 else 1e9
        doc_id = doc_ids[doc_idx]
        bi_encoder_results[qid][doc_id] = score

print("Bi-Encoder top-k results ready.")

INFO:root:Searching query embeddings in HNSW index...


Bi-Encoder top-k results ready.


In [16]:
# Print the number of retrieved documents for the first 5 queries along with their scores
print("=== Query Results ===")
for query_id, doc_scores in list(bi_encoder_results.items())[:5]:  # Limit to first 5 queries
    num_docs = len(doc_scores)  # Number of documents retrieved for this query
    print(f"Query ID: {query_id}")
    print(f"Number of retrieved documents: {num_docs}")
    print(f"Scores (truncated): {str(doc_scores)[:90]}")
    print("-" * 50)

=== Query Results ===
Query ID: 1
Number of retrieved documents: 500
Scores (truncated): {'c065954f-2019-04-18T14:32:52Z-00002-000': 4.486143324778476, '4d8487a-2019-04-18T18:20:2
--------------------------------------------------
Query ID: 2
Number of retrieved documents: 500
Scores (truncated): {'18710bc8-2019-04-18T16:37:00Z-00001-000': 3.4603981128831807, '18710bc8-2019-04-18T16:37
--------------------------------------------------
Query ID: 3
Number of retrieved documents: 500
Scores (truncated): {'82973dd4-2019-04-18T18:46:16Z-00004-000': 3.2892193728350874, '1f173b8c-2019-04-18T18:45
--------------------------------------------------
Query ID: 4
Number of retrieved documents: 500
Scores (truncated): {'1df1290b-2019-04-19T12:44:36Z-00076-000': 5.246674411330131, '8baf5944-2019-04-18T19:55:
--------------------------------------------------
Query ID: 5
Number of retrieved documents: 500
Scores (truncated): {'cf4c9cbf-2019-04-17T11:47:24Z-00043-000': 4.277390165265194, 'cf4c9cbf-20

In [17]:
# Cell 11: Evaluate Bi-Encoder Results
k_values = [5, 10, 20, 30, 100]
ndcg_bi, map_bi, recall_bi, prec_bi, mrr_bi = evaluate_results(qrels, bi_encoder_results, k_values)

print("=== Bi-Encoder (E5) Results ===")
# print("NDCG@k:", ndcg_bi)
# print("MAP:", map_bi)
# print("Recall@k:", recall_bi)
# print("Precision@k:", prec_bi)
print("NDCG@k:")
for k, v in ndcg_bi.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_bi:.4f}")

print("\nRecall@k:")
for k, v in recall_bi.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_bi.items():
    print(f"  {k:<8}: {v:.4f}")
    
print(f"\nMRR: {mrr_bi:.4f}")

=== Bi-Encoder (E5) Results ===
NDCG@k:
  NDCG@5 : 0.2941
  NDCG@10: 0.2660
  NDCG@20: 0.2727
  NDCG@30: 0.2976
  NDCG@100: 0.3739

MAP: 0.1746

Recall@k:
  Recall@5: 0.1014
  Recall@10: 0.1578
  Recall@20: 0.2375
  Recall@30: 0.2876
  Recall@100: 0.4532

Precision@k:
  P@5     : 0.2735
  P@10    : 0.2224
  P@20    : 0.1755
  P@30    : 0.1463
  P@100   : 0.0722

MRR: 0.5491


# Part III: Cross-Encoder Reranking

We'll now demonstrate how to:
1. Take the **top-k** documents from the **bi-encoder** results.
2. Rerank them with a **CrossEncoder**.
3. Evaluate again using the same `evaluate_results` function.


In [11]:
# Cell 12: Define a function to rerank with a CrossEncoder

def cross_encode_rerank(cross_encoder_model, 
                        query_ids_list, query_texts_list,
                        topk_results, corpus_text_map,
                        rerank_top_k=500, final_top_k=100):
    """
    cross_encoder_model: a CrossEncoder from sentence-transformers
    query_ids_list: list of query IDs (strings)
    query_texts_list: list of corresponding query texts
    topk_results: dict => topk_results[qid][doc_id] = some bi-encoder score
    corpus_text_map: a dict doc_id -> doc_text
    rerank_top_k: how many documents to consider for reranking from the bi-encoder results
    final_top_k: how many documents to keep after cross-encoder reranking

    returns: dict => reranked_results[qid][doc_id] = cross-encoder score
    """
    reranked_results = {}

    # Use tqdm to show total progress for all queries
    with tqdm(total=len(query_ids_list), desc="Cross-encoding queries", unit="query", ncols=80) as pbar:
        for idx, qid in enumerate(query_ids_list):
            # Get top rerank_top_k document candidates for the query
            doc_candidates = sorted(topk_results[qid].items(), key=lambda x: x[1], reverse=True)[:rerank_top_k]
            doc_ids = [doc_id for doc_id, _ in doc_candidates]
            query_text = query_texts_list[idx]

            # Prepare query-document pairs for the cross-encoder
#             pair_texts = [(f"query: {query_text}", f"passage: {corpus_text_map[doc_id]}") for doc_id in doc_ids]
            pair_texts = [(query_text, corpus_text_map[doc_id]) for doc_id in doc_ids]
            
            # Perform cross-encoder inference, ensure no additional progress bars are shown
            scores = cross_encoder_model.predict(pair_texts, show_progress_bar=False)

            # Combine document IDs with their Cross-Encoder scores
            reranked_doc_scores = {doc_ids[i]: float(scores[i]) for i in range(len(doc_ids))}

            # Sort reranked results by Cross-Encoder scores and keep final_top_k
            reranked_results[qid] = dict(sorted(reranked_doc_scores.items(), key=lambda x: x[1], reverse=True)[:final_top_k])

            # Update the progress bar
            pbar.update(1)

    return reranked_results

print('Cross-encoder rerank function defined.')

Cross-encoder rerank function defined.


### Prepare Data Structures for Reranking

- We already have `bi_encoder_results`, which is `dict[qid][doc_id] = score`.
- We need a quick way to map doc IDs to actual text for the CrossEncoder.


In [12]:
# Cell 13: Create a dictionary doc_id -> doc_text
doc_id_to_text = {}
for i, did in enumerate(doc_ids):
    doc_id_to_text[did] = doc_texts[i]

print(f"doc_id_to_text dictionary has {len(doc_id_to_text)} entries.")

doc_id_to_text dictionary has 382545 entries.


## 3.1: MiniLM CrossEncoder Reranking

### Load a CrossEncoder
Choose any cross-encoder checkpoint from Hugging Face or sentence-transformers.
For demonstration, we'll pick something like `'cross-encoder/ms-marco-MiniLM-L-6-v2'`.


In [20]:
# Cell 14: CrossEncoder initialization
cross_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"  # or any other suitable model
cross_encoder = CrossEncoder(cross_model_name, device=device)
print("CrossEncoder loaded.")

CrossEncoder loaded.


### Rerank the top-k results from the Bi-Encoder

We'll use the cross-encoder function defined above and pass in the top-k results from the Bi-Encoder.


In [23]:
# Cell 15: Rerank top-k with CrossEncoder

# Parameters to control the number of documents to rerank and return
rerank_top_k = 300  # Number of documents to consider for reranking
final_top_k = 100    # Number of documents to keep after reranking

logging.info(f"Reranking top {rerank_top_k} candidates with cross-encoder, keeping top {final_top_k}...")

cross_encoder_results_minilm = cross_encode_rerank(
    cross_encoder_model=cross_encoder,
    query_ids_list=filtered_query_ids,
    query_texts_list=filtered_query_texts,
    topk_results=bi_encoder_results,  # Bi-Encoder results
    corpus_text_map=doc_id_to_text,   # Document text mapping
    rerank_top_k=rerank_top_k,        # Candidates to rerank
    final_top_k=final_top_k           # Final results to keep
)

print("Cross-encoder reranking complete.")

INFO:root:Reranking top 300 candidates with cross-encoder, keeping top 100...
Cross-encoding queries: 100%|████████████████| 49/49 [01:00<00:00,  1.24s/query]

Cross-encoder reranking complete.


### Evaluate Cross-Encoder Reranked Results


In [24]:
# Cell 16: Evaluate the cross-encoder-based results
k_values = [5, 10, 20, 30, 100]
ndcg_ce, map_ce, recall_ce, prec_ce, mrr_ce = evaluate_results(qrels, cross_encoder_results_minilm, k_values)

# print("=== Cross-Encoder (Rerank) Results ===")
# print("NDCG@k:", ndcg_ce)
# print("MAP:", map_ce)
# print("Recall@k:", recall_ce)
# print("Precision@k:", prec_ce)

# Print formatted results
print("=== MiniLM Cross-Encoder (Rerank) Results ===")
print("NDCG@k:")
for k, v in ndcg_ce.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_ce:.4f}")

print("\nRecall@k:")
for k, v in recall_ce.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_ce.items():
    print(f"  {k:<8}: {v:.4f}")
    
print(f"\nMRR: {mrr_ce:.4f}")

=== MiniLM Cross-Encoder (Rerank) Results ===
NDCG@k:
  NDCG@5 : 0.3659
  NDCG@10: 0.3268
  NDCG@20: 0.3279
  NDCG@30: 0.3441
  NDCG@100: 0.4190

MAP: 0.2091

Recall@k:
  Recall@5: 0.1361
  Recall@10: 0.2065
  Recall@20: 0.2959
  Recall@30: 0.3369
  Recall@100: 0.4952

Precision@k:
  P@5     : 0.3918
  P@10    : 0.3000
  P@20    : 0.2204
  P@30    : 0.1714
  P@100   : 0.0810

MRR: 0.5989


## 3.2: Electra CrossEncoder Reranking
To compare the performance of different cross-encoders, here we use `'cross-encoder/ms-marco-electra-base'`.


In [25]:
# Cell 14: CrossEncoder initialization
cross_model_name = "cross-encoder/ms-marco-electra-base"  # or any other suitable model
cross_encoder = CrossEncoder(cross_model_name, device=device)
print("CrossEncoder loaded.")

CrossEncoder loaded.


In [26]:
# Cell 15: Rerank top-k with CrossEncoder

# Parameters to control the number of documents to rerank and return
rerank_top_k = 300  # Number of documents to consider for reranking
final_top_k = 100    # Number of documents to keep after reranking

logging.info(f"Reranking top {rerank_top_k} candidates with cross-encoder, keeping top {final_top_k}...")

cross_encoder_results_electra = cross_encode_rerank(
    cross_encoder_model=cross_encoder,
    query_ids_list=filtered_query_ids,
    query_texts_list=filtered_query_texts,
    topk_results=bi_encoder_results,  # Bi-Encoder results
    corpus_text_map=doc_id_to_text,   # Document text mapping
    rerank_top_k=rerank_top_k,        # Candidates to rerank
    final_top_k=final_top_k           # Final results to keep
)

print("Cross-encoder reranking complete.")

INFO:root:Reranking top 300 candidates with cross-encoder, keeping top 100...
Cross-encoding queries: 100%|████████████████| 49/49 [07:28<00:00,  9.15s/query]

Cross-encoder reranking complete.


In [28]:
# Cell 16: Evaluate the cross-encoder-based results
k_values = [5, 10, 20, 30, 100]
ndcg_ce, map_ce, recall_ce, prec_ce, mrr_ce = evaluate_results(qrels, cross_encoder_results_electra, k_values)

# print("=== Cross-Encoder (Rerank) Results ===")
# print("NDCG@k:", ndcg_ce)
# print("MAP:", map_ce)
# print("Recall@k:", recall_ce)
# print("Precision@k:", prec_ce)

# Print formatted results
print("=== Electra Cross-Encoder (Rerank) Results ===")
print("NDCG@k:")
for k, v in ndcg_ce.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_ce:.4f}")

print("\nRecall@k:")
for k, v in recall_ce.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_ce.items():
    print(f"  {k:<8}: {v:.4f}")
    
print(f"\nMRR: {mrr_ce:.4f}")

=== Electra Cross-Encoder (Rerank) Results ===
NDCG@k:
  NDCG@5 : 0.3689
  NDCG@10: 0.3506
  NDCG@20: 0.3373
  NDCG@30: 0.3553
  NDCG@100: 0.4380

MAP: 0.2100

Recall@k:
  Recall@5: 0.1222
  Recall@10: 0.2231
  Recall@20: 0.2933
  Recall@30: 0.3380
  Recall@100: 0.5141

Precision@k:
  P@5     : 0.3714
  P@10    : 0.3184
  P@20    : 0.2214
  P@30    : 0.1741
  P@100   : 0.0855

MRR: 0.6749


## 3.3: Tiny-BERT CrossEncoder Reranking
To compare the performance of different cross-encoders, here we use `'cross-encoder/ms-marco-TinyBERT-L-2-v2'`.


In [13]:
# Cell 14: CrossEncoder initialization
cross_model_name = "cross-encoder/ms-marco-TinyBERT-L-2-v2"  # or any other suitable model
cross_encoder = CrossEncoder(cross_model_name, device=device)
print("CrossEncoder loaded.")

CrossEncoder loaded.


In [14]:
# Cell 15: Rerank top-k with CrossEncoder

# Parameters to control the number of documents to rerank and return
rerank_top_k = 300  # Number of documents to consider for reranking
final_top_k = 100    # Number of documents to keep after reranking

logging.info(f"Reranking top {rerank_top_k} candidates with cross-encoder, keeping top {final_top_k}...")

cross_encoder_results_tinybert = cross_encode_rerank(
    cross_encoder_model=cross_encoder,
    query_ids_list=filtered_query_ids,
    query_texts_list=filtered_query_texts,
    topk_results=bi_encoder_results,  # Bi-Encoder results
    corpus_text_map=doc_id_to_text,   # Document text mapping
    rerank_top_k=rerank_top_k,        # Candidates to rerank
    final_top_k=final_top_k           # Final results to keep
)

print("Cross-encoder reranking complete.")

INFO:root:Reranking top 300 candidates with cross-encoder, keeping top 100...
Cross-encoding queries: 100%|████████████████| 49/49 [00:06<00:00,  7.57query/s]

Cross-encoder reranking complete.


In [15]:
# Cell 16: Evaluate the cross-encoder-based results
k_values = [5, 10, 20, 30, 100]
ndcg_ce, map_ce, recall_ce, prec_ce, mrr_ce = evaluate_results(qrels, cross_encoder_results_tinybert, k_values)

# print("=== Cross-Encoder (Rerank) Results ===")
# print("NDCG@k:", ndcg_ce)
# print("MAP:", map_ce)
# print("Recall@k:", recall_ce)
# print("Precision@k:", prec_ce)

# Print formatted results
print("=== Tiny-BERT Cross-Encoder (Rerank) Results ===")
print("NDCG@k:")
for k, v in ndcg_ce.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_ce:.4f}")

print("\nRecall@k:")
for k, v in recall_ce.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_ce.items():
    print(f"  {k:<8}: {v:.4f}")
    
print(f"\nMRR: {mrr_ce:.4f}")

=== Tiny-BERT Cross-Encoder (Rerank) Results ===
NDCG@k:
  NDCG@5 : 0.3131
  NDCG@10: 0.2998
  NDCG@20: 0.3047
  NDCG@30: 0.3222
  NDCG@100: 0.3981

MAP: 0.1848

Recall@k:
  Recall@5: 0.1132
  Recall@10: 0.1908
  Recall@20: 0.2759
  Recall@30: 0.3201
  Recall@100: 0.4827

Precision@k:
  P@5     : 0.3184
  P@10    : 0.2735
  P@20    : 0.2061
  P@30    : 0.1639
  P@100   : 0.0786

MRR: 0.5715


# Part IV: Evaluating Cross-Encoders Fine-Tuned on Other Datasets
This section demonstrates the evaluation of Cross-Encoders fine-tuned on different datasets, applied to the current dataset for relevance scoring. It includes functions to utilize pre-fine-tuned Cross-Encoder models to test their performance on the current dataset. The results provide insights into how well Cross-Encoders fine-tuned on external datasets generalize to new tasks and domains, specifically for reranking tasks or other applications requiring precise query-document matching.

## 4.1 Evaluation of Cross-Encoders Fine-Tuned using Scifact

In [18]:
#######################################################
# Rerank with the Fine-Tuned CrossEncoder
#######################################################

logging.info("Reranking top-k with the cross-encoder fine-tuned using scifact...")
fine_tuned_model_path = "./models/scifact_fine_tuned_cross_encoder_minilm"
fine_tuned_cross_encoder = CrossEncoder(fine_tuned_model_path)

rerank_top_k = 300
final_top_k = 100

cross_encoder_results_finetuned = cross_encode_rerank(
    cross_encoder_model=fine_tuned_cross_encoder,
    query_ids_list=filtered_query_ids,
    query_texts_list=filtered_query_texts,
    topk_results=bi_encoder_results,   # from your bi-encoder retrieval
    corpus_text_map=doc_id_to_text,
    rerank_top_k=rerank_top_k,
    final_top_k=final_top_k
)

print("Fine-tuned CrossEncoder reranking complete.")

INFO:root:Reranking top-k with the cross-encoder fine-tuned using scifact...
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda
Cross-encoding queries: 100%|████████████████| 49/49 [01:00<00:00,  1.24s/query]

Fine-tuned CrossEncoder reranking complete.


In [20]:
#######################################################
# Evaluate the Fine-Tuned CrossEncoder Results
#######################################################
# 1) Use the same evaluate_results function
k_values = [5, 10, 20, 30, 100]
ndcg_ft, map_ft, recall_ft, prec_ft, mrr_ft = evaluate_results(qrels, cross_encoder_results_finetuned, k_values)

# 2) Print or log the metrics
print("=== Scifact Fine-Tuned Cross-Encoder (Mini-LM) Results ===")
print("NDCG@k:")
for k, v in ndcg_ft.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_ft:.4f}")

print("\nRecall@k:")
for k, v in recall_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print(f"\nMRR: {mrr_ft:.4f}")

=== Scifact Fine-Tuned Cross-Encoder (Mini-LM) Results ===
NDCG@k:
  NDCG@5 : 0.3700
  NDCG@10: 0.3428
  NDCG@20: 0.3510
  NDCG@30: 0.3696
  NDCG@100: 0.4306

MAP: 0.2166

Recall@k:
  Recall@5: 0.1377
  Recall@10: 0.2164
  Recall@20: 0.3127
  Recall@30: 0.3576
  Recall@100: 0.4897

Precision@k:
  P@5     : 0.3837
  P@10    : 0.3061
  P@20    : 0.2367
  P@30    : 0.1830
  P@100   : 0.0802

MRR: 0.5955


In [21]:
#######################################################
# Rerank with the Fine-Tuned CrossEncoder
#######################################################

logging.info("Reranking top-k with the cross-encoder fine-tuned using scifact...")
fine_tuned_model_path = "./models/scifact_fine_tuned_cross_encoder_tinybert"
fine_tuned_cross_encoder = CrossEncoder(fine_tuned_model_path)

rerank_top_k = 300
final_top_k = 100

cross_encoder_results_finetuned = cross_encode_rerank(
    cross_encoder_model=fine_tuned_cross_encoder,
    query_ids_list=filtered_query_ids,
    query_texts_list=filtered_query_texts,
    topk_results=bi_encoder_results,   # from your bi-encoder retrieval
    corpus_text_map=doc_id_to_text,
    rerank_top_k=rerank_top_k,
    final_top_k=final_top_k
)

print("Fine-tuned CrossEncoder reranking complete.")

INFO:root:Reranking top-k with the cross-encoder fine-tuned using scifact...
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda
Cross-encoding queries: 100%|████████████████| 49/49 [00:06<00:00,  7.37query/s]

Fine-tuned CrossEncoder reranking complete.


In [22]:
#######################################################
# Evaluate the Fine-Tuned CrossEncoder Results
#######################################################
# 1) Use the same evaluate_results function
k_values = [5, 10, 20, 30, 100]
ndcg_ft, map_ft, recall_ft, prec_ft, mrr_ft = evaluate_results(qrels, cross_encoder_results_finetuned, k_values)

# 2) Print or log the metrics
print("=== Scifact Fine-Tuned Cross-Encoder (Tiny-BERT) Results ===")
print("NDCG@k:")
for k, v in ndcg_ft.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_ft:.4f}")

print("\nRecall@k:")
for k, v in recall_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print(f"\nMRR: {mrr_ft:.4f}")

=== Scifact Fine-Tuned Cross-Encoder (Tiny-BERT) Results ===
NDCG@k:
  NDCG@5 : 0.3658
  NDCG@10: 0.3227
  NDCG@20: 0.3237
  NDCG@30: 0.3481
  NDCG@100: 0.4118

MAP: 0.1970

Recall@k:
  Recall@5: 0.1358
  Recall@10: 0.1960
  Recall@20: 0.2806
  Recall@30: 0.3369
  Recall@100: 0.4752

Precision@k:
  P@5     : 0.3796
  P@10    : 0.2837
  P@20    : 0.2092
  P@30    : 0.1735
  P@100   : 0.0780

MRR: 0.5869


## 4.2 Evaluation of Cross-Encoders Fine-Tuned using NF-Corpus

In [23]:
#######################################################
# Rerank with the Fine-Tuned CrossEncoder
#######################################################

logging.info("Reranking top-k with the cross-encoder fine-tuned using NF-Corpus...")
fine_tuned_model_path = "./models/nfcorpus_fine_tuned_cross_encoder_minilm"
fine_tuned_cross_encoder = CrossEncoder(fine_tuned_model_path)

rerank_top_k = 300
final_top_k = 100

cross_encoder_results_finetuned = cross_encode_rerank(
    cross_encoder_model=fine_tuned_cross_encoder,
    query_ids_list=filtered_query_ids,
    query_texts_list=filtered_query_texts,
    topk_results=bi_encoder_results,   # from your bi-encoder retrieval
    corpus_text_map=doc_id_to_text,
    rerank_top_k=rerank_top_k,
    final_top_k=final_top_k
)

print("Fine-tuned CrossEncoder reranking complete.")

INFO:root:Reranking top-k with the cross-encoder fine-tuned using NF-Corpus...
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda
Cross-encoding queries: 100%|████████████████| 49/49 [01:00<00:00,  1.24s/query]

Fine-tuned CrossEncoder reranking complete.


In [24]:
#######################################################
# Evaluate the Fine-Tuned CrossEncoder Results
#######################################################
# 1) Use the same evaluate_results function
k_values = [5, 10, 20, 30, 100]
ndcg_ft, map_ft, recall_ft, prec_ft, mrr_ft = evaluate_results(qrels, cross_encoder_results_finetuned, k_values)

# 2) Print or log the metrics
print("=== NF-Corpus Fine-Tuned Cross-Encoder (MiniLM) Results ===")
print("NDCG@k:")
for k, v in ndcg_ft.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_ft:.4f}")

print("\nRecall@k:")
for k, v in recall_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print(f"\nMRR: {mrr_ft:.4f}")

=== NF-Corpus Fine-Tuned Cross-Encoder (MiniLM) Results ===
NDCG@k:
  NDCG@5 : 0.2871
  NDCG@10: 0.2701
  NDCG@20: 0.2762
  NDCG@30: 0.2976
  NDCG@100: 0.3729

MAP: 0.1660

Recall@k:
  Recall@5: 0.1043
  Recall@10: 0.1721
  Recall@20: 0.2449
  Recall@30: 0.2943
  Recall@100: 0.4551

Precision@k:
  P@5     : 0.2776
  P@10    : 0.2306
  P@20    : 0.1745
  P@30    : 0.1463
  P@100   : 0.0729

MRR: 0.5231


In [25]:
#######################################################
# Rerank with the Fine-Tuned CrossEncoder
#######################################################

logging.info("Reranking top-k with the cross-encoder fine-tuned using NF-Corpus...")
fine_tuned_model_path = "./models/nfcorpus_fine_tuned_cross_encoder_tinybert"
fine_tuned_cross_encoder = CrossEncoder(fine_tuned_model_path)

rerank_top_k = 300
final_top_k = 100

cross_encoder_results_finetuned = cross_encode_rerank(
    cross_encoder_model=fine_tuned_cross_encoder,
    query_ids_list=filtered_query_ids,
    query_texts_list=filtered_query_texts,
    topk_results=bi_encoder_results,   # from your bi-encoder retrieval
    corpus_text_map=doc_id_to_text,
    rerank_top_k=rerank_top_k,
    final_top_k=final_top_k
)

print("Fine-tuned CrossEncoder reranking complete.")

INFO:root:Reranking top-k with the cross-encoder fine-tuned using NF-Corpus...
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda
Cross-encoding queries: 100%|████████████████| 49/49 [00:06<00:00,  7.46query/s]

Fine-tuned CrossEncoder reranking complete.


In [26]:
#######################################################
# Evaluate the Fine-Tuned CrossEncoder Results
#######################################################
# 1) Use the same evaluate_results function
k_values = [5, 10, 20, 30, 100]
ndcg_ft, map_ft, recall_ft, prec_ft, mrr_ft = evaluate_results(qrels, cross_encoder_results_finetuned, k_values)

# 2) Print or log the metrics
print("=== NF-Corpus Fine-Tuned Cross-Encoder (Tiny-BERT) Results ===")
print("NDCG@k:")
for k, v in ndcg_ft.items():
    print(f"  {k:<7}: {v:.4f}")

print(f"\nMAP: {map_ft:.4f}")

print("\nRecall@k:")
for k, v in recall_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print("\nPrecision@k:")
for k, v in prec_ft.items():
    print(f"  {k:<8}: {v:.4f}")

print(f"\nMRR: {mrr_ft:.4f}")

=== NF-Corpus Fine-Tuned Cross-Encoder (Tiny-BERT) Results ===
NDCG@k:
  NDCG@5 : 0.1844
  NDCG@10: 0.1816
  NDCG@20: 0.2031
  NDCG@30: 0.2269
  NDCG@100: 0.3126

MAP: 0.1132

Recall@k:
  Recall@5: 0.0579
  Recall@10: 0.1206
  Recall@20: 0.2015
  Recall@30: 0.2503
  Recall@100: 0.4326

Precision@k:
  P@5     : 0.1837
  P@10    : 0.1714
  P@20    : 0.1439
  P@30    : 0.1224
  P@100   : 0.0704

MRR: 0.4176
